# Logistic Regression Model

## Setup

In [1]:
import pandas as pd
import numpy as np
import copy
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
import pickle
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score

In [2]:
trainingdata_filenames = [
    'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle', 
    'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle', 'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w10_df.pickle', 
    'oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w15_df.pickle', 'newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w1_df.pickle', 
    'newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w3_df.pickle', 'newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w5_df.pickle',
    'newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w10_df.pickle', 'newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w15_df.pickle',
    'oldgroups/lupus/lupus_oldgroups_maxT_w1_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w3_df.pickle', 
    'oldgroups/lupus/lupus_oldgroups_maxT_w5_df.pickle', 'oldgroups/lupus/lupus_oldgroups_maxT_w10_df.pickle', 
    'oldgroups/lupus/lupus_oldgroups_maxT_w15_df.pickle', 'newgroups_noNaN/lupus/lupus_newgroups_maxT_w1_df.pickle', 
    'newgroups_noNaN/lupus/lupus_newgroups_maxT_w3_df.pickle', 'newgroups_noNaN/lupus/lupus_newgroups_maxT_w5_df.pickle',
    'newgroups_noNaN/lupus/lupus_newgroups_maxT_w10_df.pickle', 'newgroups_noNaN/lupus/lupus_newgroups_maxT_w15_df.pickle',
    'oldgroups/raynauds/raynauds_oldgroups_maxT_w1_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w3_df.pickle', 
    'oldgroups/raynauds/raynauds_oldgroups_maxT_w5_df.pickle', 'oldgroups/raynauds/raynauds_oldgroups_maxT_w10_df.pickle', 
    'oldgroups/raynauds/raynauds_oldgroups_maxT_w15_df.pickle', 'newgroups_noNaN/raynauds/raynauds_newgroups_maxT_w1_df.pickle', 
    'newgroups_noNaN/raynauds/raynauds_newgroups_maxT_w3_df.pickle', 'newgroups_noNaN/raynauds/raynauds_newgroups_maxT_w5_df.pickle', 
    'newgroups_noNaN/raynauds/raynauds_newgroups_maxT_w10_df.pickle', 'newgroups_noNaN/raynauds/raynauds_newgroups_maxT_w15_df.pickle']

## Prepare Data

### Functions to separate X_train, X_test, y_train, y_test and to normalize data

X: array of shape (n_samples, n_features)

y: array of shape (n_samples,)

In [3]:
def x_y_train_test(df):
    """
    (pd.DataFrame) -> dict
    
    Returns a dictionary that contains 5 data structures: X (pd.DataFrame), X_train, X_test, y_train, y_test (np.ndarray)
    
    """
    
    #X, y
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1:]
    
    # train, test
    n_samples = X.shape[0]
    train_samples = int(0.8*n_samples) # Keep 80% of the sample for training
    
    X_train = X.iloc[:train_samples,:]
    X_test = X.iloc[train_samples:,:]
    y_train = y.iloc[:train_samples,:]
    y_test = y.iloc[train_samples:,:]
    
    dict_toreturn = {"X": X, "X_train": X_train, "X_test": X_test, "y_train": y_train, "y_test": y_test}
    
    return dict_toreturn

In [4]:
def normalize(scaler_str, X_train, X_test):
    """
    (str, pd.DataFrame, pd.DataFrame) -> dict
    
    Returns a dictionary that contains 2 arrays (X_train_norm and X_test_norm) normalized using scaler.
    """
    
    # Define scaler
    if scaler_str == "MinMaxScaler":
        scaler = preprocessing.MinMaxScaler()
    elif scaler_str == "StandardScaler":
        scaler = preprocessing.StandardScaler()
    
    # Fit scaler to train data
    scaler.fit(X_train)
    
    # Transform data
    X_train_norm = scaler.transform(X_train)
    X_test_norm = scaler.transform(X_test)
    
    dict_toreturn = {"X_train_norm": X_train_norm, "X_test_norm": X_test_norm}
    
    return dict_toreturn

## Model

In [5]:
def train_test_logistic_regr_from_files(trainingdata_filenames):
    """
    (list)->NoneType
    
    trainingdata_filenames is a list of filenames (str). 
    A logistic regression model is trained and tested on each file's data.
    Prints the metrics.
    """
    # Iterate through all the DataFrames that contain usable data
    for trainingdata_filename in trainingdata_filenames:
        # get dataframe
        this_df = pickle.load(open(trainingdata_filename, "rb"))
        # separate x and y, train and test
        xy_dict = x_y_train_test(this_df)
        X = xy_dict["X"]
        X_train = xy_dict["X_train"]
        X_test = xy_dict["X_test"]
        y_train = xy_dict["y_train"]
        y_test = xy_dict["y_test"]

        # normalize
        try:
            x_norm_dict = normalize("StandardScaler", X_train, X_test) # Name of scaler can be changed
            X_train = x_norm_dict["X_train_norm"]
            X_test = x_norm_dict["X_test_norm"]
        except (ValueError): ### I know this is cheating, but the problem is that some dataframes are empty. I'll try to fix it later.
            print("Error with:", df_name,"\n")
            continue

        print("\nData:", trainingdata_filename)
        # Simple logistic regression
        regr = linear_model.LogisticRegression(max_iter=500)
        regr.fit(X_train, y_train.values.ravel())

        # Check which classes the model has seen during training
        print ("Class labels known to the classifier:", regr.classes_)

        # Cross Validation
        scores = cross_val_score(regr,X_train, y_train.values.ravel(),cv=5)
        print("Cross validation scores:", scores)
        
        # Get test accuracy
        test_accuracy = regr.score(X_test, y_test)
        print ("Logistic regression test accuracy:", test_accuracy)

        # Make predictions on the test dataset
        y_pred = regr.predict(X_test)

        # Check how much error the model makes for each class
        y_test_list = list(y_test["label"])
        all_class_labels=list(set(y_test_list))
        # Fill a dict with keys been class labels. Values will be the num of correct pred
        correct_pred_per_class = dict.fromkeys(all_class_labels, 0) 
        for i, true_class_label in enumerate(y_test_list):
            pred_class_label = y_pred[i]
            if int(pred_class_label) == int(true_class_label):
                correct_pred_per_class[true_class_label] += 1
        # Compute the accuracy for each class
        for class_label in all_class_labels:
            num_samples_for_this_class = y_test_list.count(class_label)
            accuracy_for_this_class = correct_pred_per_class[class_label]/num_samples_for_this_class
            print ("Class", class_label, "accuracy: ", accuracy_for_this_class)

        # Check with what certainty the model does its predictions
        #class_probability = regr.predict_proba(X_test)
        #print ("Probability that each test input falls into each output class:", class_probability, "\n")

In [6]:
def train_test_logistic_regr(this_df):
    """
    (list)->NoneType
    
    trainingdata_filenames is a list of filenames (str). 
    A logistic regression model is trained and tested on each file's data.
    Prints the metrics.
    """
    # separate x and y, train and test
    xy_dict = x_y_train_test(this_df)
    X = xy_dict["X"]
    X_train = xy_dict["X_train"]
    X_test = xy_dict["X_test"]
    y_train = xy_dict["y_train"]
    y_test = xy_dict["y_test"]

    # normalize
    x_norm_dict = normalize("StandardScaler", X_train, X_test) # Name of scaler can be changed
    X_train = x_norm_dict["X_train_norm"]
    X_test = x_norm_dict["X_test_norm"]

    print("\nData:", trainingdata_filename)
    # Simple logistic regression
    regr = linear_model.LogisticRegression(max_iter=500)
    regr.fit(X_train, y_train.values.ravel())

    # Check which classes the model has seen during training
    print ("Class labels known to the classifier:", regr.classes_)

    # Cross Validation
    scores = cross_val_score(regr,X_train, y_train.values.ravel(),cv=5)
    print("Cross validation scores:", scores)
    
    # Get test accuracy
    test_accuracy = regr.score(X_test, y_test)
    print ("Logistic regression test accuracy:", test_accuracy)

    # Make predictions on the test dataset
    y_pred = regr.predict(X_test)

    # Check how much error the model makes for each class
    y_test_list = list(y_test["label"])
    all_class_labels=list(set(y_test_list))
    # Fill a dict with keys been class labels. Values will be the num of correct pred
    correct_pred_per_class = dict.fromkeys(all_class_labels, 0) 
    for i, true_class_label in enumerate(y_test_list):
        pred_class_label = y_pred[i]
        if int(pred_class_label) == int(true_class_label):
            correct_pred_per_class[true_class_label] += 1
    # Compute the accuracy for each class
    for class_label in all_class_labels:
        num_samples_for_this_class = y_test_list.count(class_label)
        accuracy_for_this_class = correct_pred_per_class[class_label]/num_samples_for_this_class
        print ("Class", class_label, "accuracy: ", accuracy_for_this_class)

    # Check with what certainty the model does its predictions
    #class_probability = regr.predict_proba(X_test)
    #print ("Probability that each test input falls into each output class:", class_probability, "\n")

In [7]:
train_test_logistic_regr_from_files(trainingdata_filenames)


Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Cross validation scores: [0.55716163 0.55059133 0.57161629 0.33114323 0.56315789]
Logistic regression test accuracy: 0.3501577287066246
Class 0.0 accuracy:  0.06329113924050633
Class 1.0 accuracy:  0.2222222222222222
Class 2.0 accuracy:  0.7269503546099291
Class 3.0 accuracy:  0.17768595041322313
Class 4.0 accuracy:  0.2389937106918239

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Cross validation scores: [0.55592105 0.63157895 0.66118421 0.40953947 0.57990115]
Logistic regression test accuracy: 0.43026315789473685
Class 0.0 accuracy:  0.21428571428571427
Class 1.0 accuracy:  0.37267080745341613
Class 2.0 accuracy:  0.6784140969162996
Class 3.0 accuracy:  0.29381443298969073
Class 4.0 accuracy:  0.34558823529411764

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickl

## Resample the dataset to account for class imbalances a

Source: https://elitedatascience.com/imbalanced-classes

Check the distribution of classes in the training dataset

In [22]:
for trainingdata_filename in trainingdata_filenames:
    this_df = pickle.load(open(trainingdata_filename, "rb"))
    lastcol = this_df.columns[-1]
    print (trainingdata_filename,":\n",this_df[lastcol].value_counts())

oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle :
 2.0    2161
3.0    1134
1.0     806
4.0     408
0.0     246
Name: label, dtype: int64
oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle :
 2.0    1792
3.0     903
1.0     614
4.0     300
0.0     190
Name: label, dtype: int64
oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle :
 2.0    1534
3.0     780
1.0     501
4.0     234
0.0     161
Name: label, dtype: int64
oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w10_df.pickle :
 2.0    1171
3.0     589
1.0     354
4.0     160
0.0     130
Name: label, dtype: int64
oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w15_df.pickle :
 2.0    970
3.0    468
1.0    285
4.0    131
0.0    115
Name: label, dtype: int64
newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w1_df.pickle :
 0    2690
2    1027
1     781
Name: label, dtype: int64
newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w3_df.pickle :
 0    2470
2     947
1     715
Name: 

In [23]:
for trainingdata_filename in trainingdata_filenames:
    this_df = pickle.load(open(trainingdata_filename, "rb"))
    lastcol = this_df.columns[-1]
    
    # Number of samples for the majority class
    max_n_samples = this_df[lastcol].value_counts().max()
    
    # Separate observations from each class into different DataFrames
    all_class_labels = this_df[lastcol].unique()
    all_class_labels.sort()
    individual_class_df=[] # a list to store the DataFrames for each class
    for class_label in all_class_labels:
        one_class_df = this_df[this_df.loc[:,lastcol]==class_label]
    
        # Resample the minority classes with replacement, setting the number of samples to match that of the majority class.
        if one_class_df.shape[0] < max_n_samples: # if the class is a minority class
            one_class_df_upsampled = resample(one_class_df, replace=True, n_samples=max_n_samples)
            individual_class_df.append(one_class_df_upsampled)
        else: # the class is the majority class
            individual_class_df.append(one_class_df)
        
    # Combine the up-sampled minority classes DataFrames with the original majority class DataFrame
    df_upsampled = pd.concat(individual_class_df)
        
    # get dataframe
    this_df = df_upsampled.sample(frac=1)
    
    # Train model
    train_test_logistic_regr(this_df)


Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Cross validation scores: [0.51995373 0.52747253 0.53441296 0.53094274 0.52083333]
Logistic regression test accuracy: 0.5071726052753355
Class 0.0 accuracy:  0.7598152424942263
Class 1.0 accuracy:  0.3995584988962472
Class 2.0 accuracy:  0.5081967213114754
Class 3.0 accuracy:  0.17494089834515367
Class 4.0 accuracy:  0.6941176470588235

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Cross validation scores: [0.59902371 0.57949791 0.58368201 0.55826936 0.57431961]
Logistic regression test accuracy: 0.5652901785714286
Class 0.0 accuracy:  0.7435897435897436
Class 1.0 accuracy:  0.4602739726027397
Class 2.0 accuracy:  0.6077348066298343
Class 3.0 accuracy:  0.3216374269005848
Class 4.0 accuracy:  0.6827956989247311

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle
Clas

## Shuffle the labels and look at the accuracy. This accuracy will be the chance accuracy

In [8]:
# Iterate through all the DataFrames that contain usable data
for trainingdata_filename in trainingdata_filenames:
    # get dataframe and shuffle labels
    this_df = pickle.load(open(trainingdata_filename, "rb"))
    lastcol = this_df.columns[-1]
    this_df[lastcol] = np.random.permutation(this_df[lastcol].values)
    # separate x and y, train and test
    xy_dict = x_y_train_test(this_df)
    X = xy_dict["X"]
    X_train = xy_dict["X_train"]
    X_test = xy_dict["X_test"]
    y_train = xy_dict["y_train"]
    y_test = xy_dict["y_test"]

    # normalize
    try:
        x_norm_dict = normalize("StandardScaler", X_train, X_test) # Name of scaler can be changed
        X_train = x_norm_dict["X_train_norm"]
        X_test = x_norm_dict["X_test_norm"]
    except (ValueError): ### I know this is cheating, but the problem is that some dataframes are empty. I'll try to fix it later.
        print("Error with:", df_name,"\n")
        continue

    print("\nData:", trainingdata_filename)
    # Simple logistic regression
    shuffle_regr = linear_model.LogisticRegression(max_iter=500)
    shuffle_labels = y_train.values.ravel()
    np.random.shuffle(shuffle_labels) 
    shuffle_regr.fit(X_train, shuffle_labels)

    # Check which classes the model has seen during training
    print ("Class labels known to the classifier:", shuffle_regr.classes_)
    
    # Test model
    shuffle_test_labels = y_test.values.ravel()
    np.random.shuffle(shuffle_test_labels) 
    test_accuracy = shuffle_regr.score(X_test, y_test.values.ravel())
    print ("Logistic regression test accuracy:", test_accuracy)


Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w1_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Logistic regression test accuracy: 0.4374342797055731

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w3_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Logistic regression test accuracy: 0.45526315789473687

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w5_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Logistic regression test accuracy: 0.4454828660436137

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w10_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Logistic regression test accuracy: 0.4864864864864865

Data: oldgroups/dysautonomia/dysautonomia_oldgroups_maxT_w15_df.pickle
Class labels known to the classifier: [0. 1. 2. 3. 4.]
Logistic regression test accuracy: 0.4619289340101523

Data: newgroups_noNaN/dysautonomia/dysautonomia_newgroups_maxT_w1_df.pickle
Class labels known t